<a href="https://colab.research.google.com/github/yashsahu02/CarDocAI/blob/main/Car_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq
!apt-get install -y-qq poppler-utils tesseract-ocr
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

!pip install q fastapi uvicorn nest_asyncio pdf2image pytesseract python-multipart

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
E: Command line option '-' [from -y-qq] is not understood in combination with the other options.
(Reading database ... 121717 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.11.1) over (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
## import and start FastAPI app
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile
import threading
import time

In [ ]:
app = FastAPI()

@app.get("/")
def home():
  return {"message":"Server is Running...."}
  nest_asyncio.apply()

In [ ]:
## Simple OCR Helper
from pdf2image import convert_from_path
import pytesseract

def extract_text_from_pdf(pdf_path):
  full=""
  for p in pages:
    full += pytesseract.image_to_string(p)+"\n"
  return full

In [ ]:
from numpy import extract
## Upload API
@app.post("/upload")
async def upload_file(file: UploadFile = File(...)):
  path = f"tmp/{file.filename}"
  with open(file, "wb") as f:
    f.write(file.file.read())
  text = extract_text_from_pdf(path) ##OCR
  return {"text": text[:2000]} ## return first 2000 chars

In [ ]:
## start the server
def run():
  uvicorn.run(app, host="0.0.0.0", port=8000) # Corrected 'appp' to 'app'

# The threading and sleep should be outside the run function if run() is meant to start the server
# and then the current thread continues. If run() is supposed to be daemonized, then:
# threading.Thread(target=run, daemon=True).start()
# time.sleep(1)
# print("Server started on part 8000")
# For simplicity, let's keep it in a way that allows direct execution.

# To start the server in a separate thread and keep the current notebook cell alive:
server_thread = threading.Thread(target=run, daemon=True)
server_thread.start()
time.sleep(1) # Give the server a moment to start
print("Server started on port 8000")


INFO:     Started server process [8306]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Server started on port 8000


In [ ]:
## Public URL using CloudFlare
import subprocess, re, time
cmd=["cloudflared", "tunnel", "--url", "http://localhost:8000"]

# Corrected line: text=True should be a separate argument.
proc=subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

public_url = None
start = time.time()

while True:
  line = proc.stdout.readline()
  print(line.strip())

  m = re.search(r"https://.*trycloudflare.com", line)
  if m:
    public_url = m.group(0)
    print("\nPUBLIC URL: ", public_url)
    print("open docs here: ", public_url + "/docs")
    break

  if time.time() - start > 25:
    print("Timed out waiting for cloudflared tunnel to establish.")
    break

2025-12-03T12:34:39Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-03T12:34:39Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-03T12:34:42Z INF +--------------------------------------------------------------------------------------------+
2025-12-03T12:34:42Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-12-03T12:34:42Z INF |  https://interpretation-quiz-sponsored-cruise.trycloud